In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch
import ollama

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

In [3]:
PATH = "pdf/JavaBasico.pdf"


reader = PdfReader(PATH)

text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

In [4]:
text_split = []
metadata=[]
chunk_size = 432
chunk_overlap = 8

for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})
embeddings_db = embeddings.encode(text_split)

In [5]:
for i, d in enumerate(text_split):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadata[i]
    )

In [ ]:
while True:

    query = []

    prompt = input("Pergunte algo")
    response = embeddings.encode([prompt])
    results = collection.query(
            query_embeddings=response,
            n_results=10,
            where={"name": "book-langchain"}
        )
    prompt_template = f"Você é um chatbot especializado em perguntas e respostas. Para responder essa pergunta: {prompt}, você precisa pesquisar somente nesses dados: {query}."

    data = results['documents'][0]
    distance = results['distances']      
    
    
    
    for k in range(10):
        if distance[0][k] < 0.9:
            query.append(data[k])

    if len(query) == 0:
            prompt_template = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
    output = ollama.generate(
            model="llama2",
            prompt=prompt_template
            )
    
    output = output['response']
    print(output)
    print("\n")

Pergunte algo o que é docker?



Oh, desculpe! Eu sou um assistente de chatbot e tenho acesso limitado às informações. Peço desculpas, mas não posso fornecer informações sobre a pergunta que você fez. Meu conhecimento é limitado e não tenho acesso a informações exteriores. Pode me fazer outra pergunta?




Pergunte algo o que é incremento e decremento?



Ah, uma pergunta interessante! Incremento e decremento são termos utilizados em diversas áreas, como matemática, finanças, programação e outros. Aqui está a minha resposta baseada nas informações pesquisadas:

Incremento e decremento são termos que indicam mudanças na quantidade de algo em um determinado tempo.

* Incremento é o aumento da quantidade de algo em um determinado período de tempo. Por exemplo, se você tiver 10 moedas e adquirir 2 mais, você agora tem 12 moedas, o que é um incremento de 2 moedas.
* Decremento é a diminuição da quantidade de algo em um determinado período de tempo. Por exemplo, se você tiver 10 moedas e perder 2, você agora tem 8 moedas, o que é um decremento de 2 moedas.

Em resumo, incremento é uma mudança positiva na quantidade de algo, enquanto decremento é uma mudança negativa na quantidade de algo.

Espero que isso ajude! Se você tiver alguma outra pergunta, não hesite em perguntar. Estou aqui para ajudar.




Pergunte algo incremento e decremento na orientação a objeto



As orientações a objetos são importantes para garantir que as informações estejamOrganized and easily accessible. Here are some key points to consider when dealing with increments and decrements in object orientation:

1. Increment: An increment is an operation that adds a value to a variable or property of an object. For example, if you have a variable `age` in an object `person`, you can increment it by adding 1 year to the person's age.
2. Decrement: A decrement is the opposite of an increment. It subtracts a value from a variable or property of an object. For example, if you have a variable `health` in an object `patient`, you can decrement it by 1 point if the patient's health decreases.
3. Object properties: Increment and decrement operations can be applied to properties of objects, such as `age`, `health`, or `money`. These properties are instances of classes that define their behavior, such as `Number` for a numeric property or `String` for a text property.
4. Classes: Increme